In [3]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
cwd = Path.cwd().parent
ibactivity_processed = cwd/'data'/'ibactivity_processed'
ibactivity_processed

WindowsPath('c:/Users/Javier/Python/PortfolioPython/code/data/ibactivity_processed')

In [124]:
report = 'U9233567_20170927_20170927.csv'
report

'U9233567_20170927_20170927.csv'

In [5]:
def convert_to_df(folder,a_file):
    data = pd.read_csv(folder/a_file, names=list('abcdefghijklmnopq'))
    data = data.loc[~data.a.isin(['Codes', 'Notes/Legal Notes','Account Information','Change in NAV','Mark-to-Market Performance Summary','Realized & Unrealized Performance Summary'])]
    return data

In [6]:
def process_option_prices(data):
    date = data.loc[data.a =='Statement']
    date = date.loc[date.c =='Period']
    date.dropna(axis=1,inplace=True)
    date.drop(columns=['a','b','c'], inplace=True, errors='ignore')
    date.reset_index(inplace=True,drop=True)
    date = date.iloc[0:1]
    date.columns = ['Date']

    data_op = data.loc[data.a =='Open Positions']
    if len(data_op) > 0:
        op = data_op.loc[data_op.b == 'Data']
        if len(op) > 0:
            first_row = op.index.values[0]-1
            header_row = list(data_op.loc[first_row])
            op.columns = header_row
            op = op.loc[:,op.columns.notna()] 
            op = op.loc[(op['Asset Category'] == 'Equity and Index Options')|( op['Asset Category'] =='Options On Futures')]
            op.drop(columns= ['Open Positions','Header','DataDiscriminator','Asset Category','Quantity','Mult','Cost Price','Cost Basis','Value','Unrealized P/L','Code'],inplace=True, errors='ignore')
            op = pd.concat([date,op],axis=1).fillna( method='ffill').dropna()
        else:
            op = pd.DataFrame()
    else:
        op = ca = pd.DataFrame()
    return op

In [7]:
def get_all_option_prices(folder):
    #Creates a list of all files in folder
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    op_list = []
    #Iterates through all files in filelist
    for file_ in filelist:
        #Checks if file is a .csv
        if file_.lower().endswith('.csv'):
            data = convert_to_df(folder,file_)
            op = process_option_prices(data)
            #if type(div) != 'NonType':
            if len(op) !=0:
                op_list.append(op)
    if len(op_list) == 0:
        option_prices = pd.DataFrame()
    else:
        option_prices = pd.concat(op_list,sort=False )
    return option_prices

In [132]:
data = convert_to_df(ibactivity_processed, report)

In [139]:
op = process_option_prices(data)

In [140]:
op

,Date,Currency,Symbol,Close Price
120,"September 27, 2017",USD,AAPL 16FEB18 165.0 C,5.2612
121,"September 27, 2017",USD,AAPL 16FEB18 165.0 P,16.0998
122,"September 27, 2017",USD,AMD 17NOV17 15.0 C,0.4057
123,"September 27, 2017",USD,ATVI 27OCT17 61.0 P,0.938
124,"September 27, 2017",USD,IBKR 15DEC17 39.0 P,0.2341
125,"September 27, 2017",USD,IBKR 15DEC17 41.0 C,4.0011
126,"September 27, 2017",USD,MOMO 20OCT17 35.0 P,2.9811
127,"September 27, 2017",USD,SSTK 17NOV17 30.0 P,0.6292
128,"September 27, 2017",USD,SWKS 17NOV17 92.5 P,1.7389
129,"September 27, 2017",USD,VIX 20DEC17 11.0 P,0.4065


In [8]:
op = get_all_option_prices(ibactivity_processed)
op

,Date,Currency,Symbol,Close Price
53,"March 10, 2017",USD,WFM 19MAY17 28.0 P,1.0027
50,"March 13, 2017",USD,WFM 19MAY17 28.0 P,1.0894
55,"March 14, 2017",USD,MTCH 21APR17 17.5 P,1.1994
56,"March 14, 2017",USD,WFM 19MAY17 28.0 P,1.1571
52,"March 15, 2017",USD,MTCH 21APR17 17.5 P,1.1662
...,...,...,...,...
951,"October 31, 2019",USD,TTD 21JAN22 180.0 P,52.2247
952,"October 31, 2019",USD,UVXY 18JUN21 21.0 P,11.145
953,"October 31, 2019",USD,UVXY 21JAN22 50.0 C,6.6677
954,"October 31, 2019",USD,VIX 19FEB20 20.0 P,3.7902
